# Load Data

In [ ]:
import pickle
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
 with open('../data/processed/rdp_ds/adj_mat.dat', 'rb')  as f:
    adj_mat = pickle.load(f)

with open('../data/processed/rdp_ds/adj_mat_ind_station_mapper.dat', 'rb') as f:
    ind_station_mapper = pickle.load(f)

with open('../data/processed/rdp_ds/speeds.dat', 'rb')  as f:
    speed_df = pickle.load(f)

**Get Time Series for Station with Most Data**

In [ ]:
station_speed = speed_df[speed_df.apply(lambda x: x.isna().sum()).idxmin()]
station_speed = station_speed.fillna(method='ffill') # ffill since only 2 missing values. we could also drop these
station_speed = station_speed[station_speed.index.month == 6] # subset and choose data in june
station_speed

In [ ]:
fig = px.line(x=station_speed.index, y=station_speed, title='Time Series Plot')
fig.update_xaxes(title='Time')
fig.update_yaxes(title='Speed (mph)')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
fig = seasonal_decompose(station_speed, period=5*12*24).plot()
fig.set_size_inches(13, 8)

We clearly see seasonality the same time each day during night time. We will remove this by taking the difference of the 288 (daily) lag.

In [ ]:
import numpy as np

In [ ]:
station_speed_no_seasonal = (station_speed - station_speed.shift(288)).dropna()
fig = px.line(x=station_speed_no_seasonal.index, y=station_speed_no_seasonal, title='Time Series Plot w/ Seasonal Component Removed')
fig.update_xaxes(title='Time')
fig.update_yaxes(title='')

In [ ]:
station_speed_no_seasonal_stationary = (station_speed_no_seasonal - station_speed_no_seasonal.shift(1)).dropna()
fig = px.line(x=station_speed_no_seasonal_stationary.index, y=station_speed_no_seasonal_stationary, title='Time Series Plot w/ Seasonal Component Removed and First Difference')
fig.update_xaxes(title='Time')
fig.update_yaxes(title='')

In [ ]:
fig = seasonal_decompose(station_speed_no_seasonal, period=5*12*24).plot()
fig.set_size_inches(13, 8)

**Test if Time Series is Stationary**

$H_0:$ The time series is non-stationary.
<br>
$H_1:$ The time series is stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
test_stat, p, _, _ , _, _ = adfuller(station_speed_no_seasonal_stationary.dropna())
p

We can conclude the time series is stationary.

**Build Auto ARIMA Model**

In [ ]:
cutoff = int(station_speed.shape[0] * 0.75)
differencing = station_speed.shift(288) + station_speed.shift(1)
train = (station_speed - differencing).iloc[:cutoff].dropna()
test = station_speed.iloc[cutoff:]

In [ ]:
import pmdarima as pmd

In [ ]:
def arimamodel(timeseriesarray):
    autoarima_model = pmd.auto_arima(timeseriesarray, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              trace=True)
    return autoarima_model

arima_model = arimamodel(train)
arima_model.summary()

In [ ]:
# with open('./trained/ARIMA/arima(0,0,1).dat', 'wb') as f:
#     pickle.dump(arima_model, f)

Evaluate:

In [ ]:
preds, conf = arima_model.predict(test.shape[0], return_conf_int=True, alpha=0.05)

conf = pd.DataFrame(conf).rename(columns={0: 'lower', 1: 'upper'})
conf['diff'] = differencing.iloc[cutoff:].values
conf = conf.apply(lambda x: [x['lower'] + x['diff'], x['upper'] + x['diff']], axis=1).apply(pd.Series).values

preds = preds + differencing.iloc[cutoff:]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=test.index, y=test, name='True Values'))
fig.add_trace(go.Line(x=test.index, y=preds, name='Predicted Values'))
fig.update_layout(
    title="ARIMA(0,0,1) Forecast Results",
    xaxis_title="Time",
    yaxis_title="Forecast")
fig.add_traces([go.Scatter(x =test.index, y=conf[:, 1],
                    mode = 'lines', line_color = 'rgba(0,0,0,0)',
                    showlegend = False),
                go.Scatter(x=test.index, y=conf[:,0],
                    mode = 'lines', line_color = 'rgba(0,0,0,0)',
                    name = '95% CI',
                    fill='tonexty', fillcolor = 'rgba(255, 0, 0, 0.2)')])


In [ ]:
# fig.write_html('../plots/ARIMA(0,0,1).html')

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# with open('./trained/ARIMA/metrics_ARIMA(0,0,1).dat', 'wb') as f:
#     metrics = {'mse': mean_squared_error(test, preds), 'rmse': mean_squared_error(test, preds, squared=False), 'r2': r2_score(test, preds), 'aic': arima_model.aic()}
#     pickle.dump(metrics, f)

**Build ARIMA Based on PACF and ACF**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(18,12))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(station_speed_no_seasonal_stationary, lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(station_speed_no_seasonal_stationary, lags=50, ax=ax2)

PACF suggests 2 significant lags in the AR model. ACF suggests 2 significant lags in MA model.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
order = (2, 0, 2)
arima_model_custom = ARIMA(train, order=order)
arima_model_custom = arima_model_custom.fit()


In [ ]:
arima_model_custom.summary()

In [ ]:
# with open(f'./trained/ARIMA/arima({order[0]},{order[1]},{order[2]}).dat', 'wb') as f:
#     pickle.dump(arima_model_custom, f)

Evaluate:

In [ ]:
preds, std_err, conf = arima_model_custom.forecast(test.shape[0], alpha=0.05)

conf = pd.DataFrame(conf).rename(columns={0: 'lower', 1: 'upper'})
conf['diff'] = differencing.iloc[cutoff:].values
conf = conf.apply(lambda x: [x['lower'] + x['diff'], x['upper'] + x['diff']], axis=1).apply(pd.Series).values

preds = preds + differencing.iloc[cutoff:]


In [ ]:
fig = go.Figure()
fig.add_trace(go.Line(x=test.index, y=test, name='True Values'))
fig.add_trace(go.Line(x=test.index, y=preds, name='Predicted Values'))
fig.update_layout(
    title=f"ARIMA({order[0]},{order[1]},{order[2]}) Forecast Results",
    xaxis_title="Time",
    yaxis_title="Forecast")
fig.add_traces([go.Scatter(x =test.index, y=conf[:, 1],
                    mode = 'lines', line_color = 'rgba(0,0,0,0)',
                    showlegend = False),
                go.Scatter(x=test.index, y=conf[:,0],
                    mode = 'lines', line_color = 'rgba(0,0,0,0)',
                    name = '95% CI',
                    fill='tonexty', fillcolor = 'rgba(255, 0, 0, 0.2)')])


In [ ]:
# fig.write_html(f'../plots/ARIMA({order[0]},{order[1]},{order[2]}).html')

In [ ]:
# with open(f'./trained/ARIMA/metrics_ARIMA({order[0]},{order[1]},{order[2]}).dat', 'wb') as f:
#     metrics = {'mse': mean_squared_error(test, preds), 'rmse': mean_squared_error(test, preds, squared=False), 'r2': r2_score(test, preds), 'aic': arima_model_custom.aic}
#     pickle.dump(metrics, f)